In [13]:
!pip install sentence-transformers

In [14]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv
/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv
/kaggle/input/youtube-videos-title-description-comments/UScomments.csv
/kaggle/input/youtube-videos-title-description-comments/GB_category_id.json
/kaggle/input/youtube-videos-title-description-comments/US_category_id.json
/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv
/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv
/kaggle/input/youtube-videos-title-description-comments/USvideos.csv


In [15]:
gb_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv', on_bad_lines='skip')
gb_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [16]:
gb_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv')
gb_text_df = gb_text_df.drop('Unnamed: 0', axis=1)
gb_text_df.head()

,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [17]:
gb_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv', on_bad_lines='skip')
gb_comments_df.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [18]:
us_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/USvideos.csv', on_bad_lines='skip')
us_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [19]:
us_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv')
us_text_df = us_text_df.drop('Unnamed: 0', axis=1)
us_text_df.head()

,id,title,description
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Grab your 365 DAY MAVERICK MERCH! Exclusively ...
1,cLdxuaxaQwc,My Response,NaN
2,WYYvHb03Eog,Apple iPhone X first look,The Apple iPhone X is the new super flagship p...
3,sjlHnJvXdQs,iPhone X (parody),MORE APPLE STUFF ► https://www.youtube.com/wat...
4,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,SUBSCRIBE: http://bit.ly/A24subscribe\n\nBased...


In [20]:
us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')
us_comments_df.head()

/tmp/ipykernel_35/3472227992.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [21]:
text_df = pd.concat([gb_text_df, us_text_df], axis=0)
print("Len of GB data - ", len(gb_text_df))
print("Len of GB data - ", len(us_text_df))
print("Len of combined data - ", len(text_df))
text_df.head()

Len of GB data -  7120
Len of GB data -  7227
Len of combined data -  14347


,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [22]:
comments_df = pd.concat([gb_comments_df, us_comments_df], axis=0)
print("Len of GB data - ", len(gb_comments_df))
print("Len of GB data - ", len(us_comments_df))
print("Len of combined data - ", len(comments_df))
comments_df.head()

Len of GB data -  718452
Len of GB data -  691400
Len of combined data -  1409852


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [23]:
details_df = pd.concat([gb_detail_df, us_detail_df], axis=0)
print("Len of GB data - ", len(gb_detail_df))
print("Len of GB data - ", len(us_detail_df))
print("Len of combined data - ", len(details_df))
details_df.head()

Len of GB data -  7993
Len of GB data -  7992
Len of combined data -  15985


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09
